In [1]:
import cv2
import numpy as np
from skimage.feature import hog
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
import pickle


In [2]:
def resize_image(img , width , height):
    return cv2.resize(img , (width , height))

In [3]:
def extract_hand_segment(image):
    # Convert the input BGR image to the YCrCb color space
    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Split the channels of the YCrCb image into Y, Cr, and Cb channels
    y_channel, cr_channel, cb_channel = cv2.split(ycrcb_image)

    # Thresholding to create a binary mask for the hand region based on Cr and Cb channels
    _, cr_mask = cv2.threshold(cr_channel, 133, 173, cv2.THRESH_BINARY)
    _, cb_mask = cv2.threshold(cb_channel, 77, 127, cv2.THRESH_BINARY)
    hand_mask = cv2.bitwise_and(cr_mask, cb_mask)

    # Dilate the hand mask to fill in any gaps
    kernel = np.ones((1, 1), np.uint8)
    dilated_mask = cv2.dilate(hand_mask, kernel, iterations=1)

    # Apply a closing operation to fill any black gaps in the hand
    closed_mask = cv2.morphologyEx(dilated_mask, cv2.MORPH_CLOSE, kernel)

    # Find the contours of the object in the mask
    _, contours, hierarchy = cv2.findContours(closed_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour (which should be the hand)
    contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
    largest_contour = max(contour_sizes, key=lambda x: x[0])[1]

    # Create a mask of the hand contour
    hand_mask = np.zeros_like(dilated_mask)
    cv2.drawContours(hand_mask, [largest_contour], 0, 255, cv2.FILLED)

    # Apply erosion to reduce the size of the segmented area
    kernel = np.ones((1, 1), np.uint8)
    eroded_mask = cv2.erode(hand_mask, kernel, iterations=1)

    # Apply the hand mask to the original image to extract the hand
    hand_segment = cv2.bitwise_and(image, image, mask=eroded_mask)

    # Return the segmented hand image
    return hand_segment

In [4]:
def apply_closing(image, kernel_size=(150, 150)):
    # Convert the image to YCrCb color space
    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Split the channels
    _, cr_channel, cb_channel = cv2.split(ycrcb_image)

    # Thresholding to create a binary mask for hand region
    _, cr_mask = cv2.threshold(cr_channel, 133, 173, cv2.THRESH_BINARY)
    _, cb_mask = cv2.threshold(cb_channel, 77, 127, cv2.THRESH_BINARY)

    # Combine the masks
    mask = cv2.bitwise_and(cr_mask, cb_mask)

    # Apply morphological closing to fill gaps in the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    mask_closed = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Apply the mask to the original image to extract the hand region
    hand_segment = cv2.bitwise_and(image, image, mask=mask_closed)

    # Return the segmented hand image
    return hand_segment

In [5]:
# Define the image transformation functions
def flip(image):
    # Flip the image horizontally
    return cv2.flip(image, 1)

def rotate(image):
    # Rotate the image by 30 degrees
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), 180, 1)
    return cv2.warpAffine(image, M, (cols, rows))

In [6]:
def preProcessing(image):
    image = resize_image(image , 4*128 , 4*64)
    image = extract_hand_segment(image)
    # image = apply_closing(image)    
    image = resize_image(image , 128 , 64)
    return image

In [7]:
def augmentation(img):
    images = []

    # Rotate the image
    rotated = rotate(img)
    images.append(rotated)

    # Flip the rotated image
    flipped = flip(img)
    images.append(flipped)

    rotatedFlipped = flip(rotated)
    images.append(rotatedFlipped)

    return images

In [8]:
# Prepare the training data
X = []
y = []
images = []
for i in range(0, 6):
    for j in range(1, 180):
        filename = 'men/{}/{}_men ({}).JPG'.format(i, i , j)
        img = cv2.imread(filename)
        if  img is None:
            continue    
        
        
        # Preprocess the image
        finalImg = preProcessing(img)

        # Save the final image to array of images
        images.append(finalImg)

        # Save the label of the image
        y.append(i)
        

        # Make new image of the preprocessed image 
        augmentedImages = augmentation(finalImg)

        # Save the new image to array of images
        images.append(augmentedImages[0])
        images.append(augmentedImages[1])
        images.append(augmentedImages[2])

        # Save the label of the image
        y.append(i)
        y.append(i)
        y.append(i)


        filename = 'Woman/{}/{}_woman ({}).JPG'.format(i, i, j)
        img = cv2.imread(filename)
        if  img is None:
            continue    
        

        # Preprocess the image
        finalImg = preProcessing(img)

        # Save the final image to array of images
        images.append(finalImg)

        # Save the label of the image
        y.append(i)
        

        # Make new image of the preprocessed image 
        augmentedImages = augmentation(finalImg)

        # Save the new image to array of images
        images.append(augmentedImages[0])
        images.append(augmentedImages[1])
        images.append(augmentedImages[2])

        # Save the label of the image
        y.append(i)
        y.append(i)
        y.append(i)

        


In [9]:
# Extract hog features from the preprocessed images and save it.
hog_features = hog(images[0], orientations=12, pixels_per_cell=(12, 12), cells_per_block=(3, 3), block_norm='L2-Hys')
X = np.zeros((len(y) , len(hog_features)))
for i in range(len(y)):
     hog_features = hog(images[i], orientations=12, pixels_per_cell=(12, 12), cells_per_block=(3, 3), block_norm='L2-Hys')
     X[i] = (np.array(hog_features))   

In [10]:
# Split the data to training data and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Try different svm kernels to find the best one for this application.
kernel= ['linear', 'poly', 'rbf', 'sigmoid']
for i in kernel:
    # Train the SVM model
    model = svm.SVC(kernel=i, C=100, gamma=0.1, degree=6)
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f'Accuracy of {i} kernel: {accuracy*100}%')

Accuracy of linear kernel: 70.46885934219735%
Accuracy of poly kernel: 84.53463960811757%
Accuracy of rbf kernel: 84.1147655703289%
Accuracy of sigmoid kernel: 26.172148355493352%


In [12]:
# Fit the data into the best model.
model = svm.SVC(kernel='poly', C=100, gamma=0.1, degree=6)
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print(f'Accuracy of poly kernel: {accuracy*100}%')

Accuracy of poly kernel: 84.53463960811757%


In [13]:
# Save the model to a file.
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))